In [1]:
# Concepts
# Must to know

# 1. Paged Attention
# 2. Continuous Batching
# 3. Prefix Caching
# 4. Speculative Decoding

# Next Level

# 5. Multi-GPU (Tensor/Pipelinine Parallelism)
# 6. Multi-Node Dynamic Serving (Data Paralellism + Load Balancing)

In [2]:
%%capture 
!pip install vllm

In [4]:
# Runpod Pytorch 2.2.0 2 RTX 4090 (24GB)
# run in terminal
# vllm serve facebook/opt-125m --host 0.0.0.0 --port 8000

In [12]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"facebook/opt-125m","object":"model","created":1764881333,"owned_by":"vllm","root":"facebook/opt-125m","parent":null,"max_model_len":2048,"permission":[{"id":"modelperm-a387a7352ae23efc","object":"model_permission","created":1764881333,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [11]:
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:8000/v1", api_key="EMPTY")

models = client.models.list()
print([m.id for m in models.data])


[2025-12-04 20:48:47] INFO _client.py:1025: HTTP Request: GET http://127.0.0.1:8000/v1/models "HTTP/1.1 200 OK"


['facebook/opt-125m']


In [20]:
!nvidia-smi


Thu Dec  4 20:59:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:81:00.0 Off |                  Off |
|  0%   25C    P8             23W /  450W |       1MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
os.environ["VLLM_USE_V1"] = "0"

from vllm import LLM
llm = LLM("facebook/opt-125m")


INFO 12-04 21:02:41 [utils.py:253] non-default args: {'seed': None, 'disable_log_stats': True, 'model': 'facebook/opt-125m'}
WARNING 12-04 21:02:41 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.
INFO 12-04 21:02:42 [model.py:637] Resolved architecture: OPTForCausalLM
INFO 12-04 21:02:42 [model.py:1750] Using max model len 2048
INFO 12-04 21:02:42 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:43 [core.py:93] Initializing a V1 LLM engine (v0.12.0) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, qua

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=4095) INFO 12-04 21:02:46 [default_loader.py:308] Loading weights took 0.25 seconds
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:47 [gpu_model_runner.py:3549] Model loading took 0.2389 GiB memory and 1.312258 seconds
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:48 [backends.py:655] Using cache directory: /root/.cache/vllm/torch_compile_cache/0d7aac6db1/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:48 [backends.py:715] Dynamo bytecode transform time: 1.49 s
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:49 [backends.py:216] Directly load the compiled graph(s) for dynamic shape from the cache, took 0.352 s
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:49 [monitor.py:34] torch.compile takes 1.84 s in total
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:50 [gpu_worker.py:359] Available KV cache memory: 20.42 GiB
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:50 [kv_cache_utils.py:1286] GPU KV cache size: 594,800 tokens
(EngineCore_DP0 pid=4095) INFO

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:00<00:00, 79.63it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:00<00:00, 89.69it/s]


(EngineCore_DP0 pid=4095) INFO 12-04 21:02:52 [gpu_model_runner.py:4466] Graph capturing finished in 2 secs, took 0.20 GiB
(EngineCore_DP0 pid=4095) INFO 12-04 21:02:52 [core.py:254] init engine (profile, create kv cache, warmup model) took 5.40 seconds
INFO 12-04 21:02:53 [llm.py:343] Supported tasks: ['generate']
ERROR 12-04 23:07:18 [core_client.py:600] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.


In [2]:
from vllm import SamplingParams

prompts = ["Hello, my name is", "The capital of France is"]
sampling_params = SamplingParams(temperature=0.8, max_tokens=50)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Prompt: {output.prompt}")
    print(f"Output: {output.outputs[0].text}")
    print("-" * 40)

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: Hello, my name is
Output:  Joel, my dad is Anton Harriman and my wife is Lydia.    I'm a bit shy, but I like to make jokes and I really love cooking. So that's why I'm really scared of you guys because we're
----------------------------------------
Prompt: The capital of France is
Output:  considered to be the capital of the empire. If the imperial capital goes to the next city, the capital of France is then called the capital.
Imperial capital is considered along with capital.
----------------------------------------


In [9]:
LLM??

Init signature:
LLM(
    model: str,
    *,
    runner: Literal['auto', 'generate', 'pooling', 'draft'] = 'auto',
    convert: Literal['auto', 'none', 'embed', 'classify', 'reward'] = 'auto',
    tokenizer: str | None = None,
    tokenizer_mode: Union[Literal['auto', 'hf', 'slow', 'mistral'], str] = 'auto',
    skip_tokenizer_init: bool = False,
    trust_remote_code: bool = False,
    allowed_local_media_path: str = '',
    allowed_media_domains: list[str] | None = None,
    tensor_parallel_size: int = 1,
    dtype: Literal['auto', 'half', 'float16', 'bfloat16', 'float', 'float32'] = 'auto',
    quantization: Optional[Literal['awq', 'deepspeedfp', 'tpu_int8', 'fp8', 'ptpc_fp8', 'fbgemm_fp8', 'fp_quant', 'modelopt', 'modelopt_fp4', 'bitblas', 'gguf', 'gptq_marlin_24', 'gptq_marlin', 'gptq_bitblas', 'awq_marlin', 'gptq', 'compressed-tensors', 'bitsandbytes', 'hqq', 'experts_int8', 'ipex', 'quark', 'moe_wna16', 'torchao', 'auto-round', 'rtn', 'inc', 'mxfp4', 'petit_nvfp4', 'cpu_gptq', 'c

In [7]:
llm.llm_engine??

Type:        LLMEngine
String form: <vllm.v1.engine.llm_engine.LLMEngine object at 0x7739a000e4a0>
File:        /usr/local/lib/python3.10/dist-packages/vllm/v1/engine/llm_engine.py
Source:     
class LLMEngine:
    """Legacy LLMEngine for backwards compatibility."""

    def __init__(
        self,
        vllm_config: VllmConfig,
        executor_class: type[Executor],
        log_stats: bool,
        aggregate_engine_logging: bool = False,
        usage_context: UsageContext = UsageContext.ENGINE_CONTEXT,
        stat_loggers: list[StatLoggerFactory] | None = None,
        mm_registry: MultiModalRegistry = MULTIMODAL_REGISTRY,
        use_cached_outputs: bool = False,
        multiprocess_mode: bool = False,
    ) -> None:
        self.vllm_config = vllm_config
        self.observability_config = vllm_config.observability_config
        self.model_config = vllm_config.model_config
        self.cache_config = vllm_config.cache_config

        self.log_stats = log_stats

        execut